# [官方案例](https://rqalpha.readthedocs.io/zh-cn/latest/intro/install.html)

## 下载数据

免费数据`rqalpha download-bundle`

默认安装路径`~/.rqalpha`  

该目录下文件：
`dividends.h5`,`ex_cum_factor.h5`,`funds.h5`,`future_info.json`,`futures.h5`,`indexes.h5`,`instruments.pk`,`share_transformation.json`,`split_factor.h5`,`st_stock_days.h5`,`stocks.h5`,`suspended_days.h5`, `trading_dates.npy`, `yield_curve.h5`  

一共10个H5文件，2个JSON文件，1个npy文件，1个pk文件。

> - h5 文件指的是 Hierarchical Data Format 5（HDF5）文件 ，即层次数据格式第 5 代版本。它是用于存储科学数据的文件格式和库文件，由美国超级计算与应用中心研发，现由非营利组织 HDF 小组提供支持。
> - npy 文件是 NumPy 库专有的二进制文件格式 ，用于高效存储和加载多维数组对象。
> - pk 文件

In [3]:
import h5py

file_path = r'C:\Users\bllos\.rqalpha\bundle\yield_curve.h5'
f = h5py.File(file_path, 'r')

for name in f:
    print(name)
    data = f[name][:]
    print(data)

f.close()

data
[(20050104, 0.027393, 0.027712, 0.028029, 0.028344, 0.029278, 0.030195, 0.031095, 0.034527, 0.037687, 0.040577, 0.043195, 0.045542, 0.047618, 0.049423, 0.050958, 0.052221, 0.05447 , 0.049944, 0.020563,      nan,      nan)
 (20050105, 0.025691, 0.025962, 0.026232, 0.026501, 0.0273  , 0.028089, 0.028868, 0.031881, 0.03473 , 0.037415, 0.039936, 0.042293, 0.044487, 0.046516, 0.048382, 0.050084, 0.056134, 0.058088, 0.049705,      nan,      nan)
 (20050106, 0.024908, 0.02521 , 0.02551 , 0.025809, 0.026699, 0.027577, 0.028443, 0.031788, 0.034943, 0.037908, 0.040683, 0.043268, 0.045663, 0.047867, 0.049882, 0.051707, 0.057979, 0.0595  , 0.048289,      nan,      nan)
 ...
 (20250428, 0.013621, 0.014112, 0.0145  , 0.014508, 0.014697, 0.0146  , 0.014601, 0.01483 , 0.015177,      nan, 0.015393,      nan, 0.016029,      nan,      nan, 0.016483, 0.018012, 0.01953 , 0.019005, 0.019982, 0.020107)
 (20250429, 0.013624, 0.014512, 0.014523, 0.01485 , 0.014784, 0.01455 , 0.01455 , 0.014616, 0.015009, 

In [5]:
import pandas as pd
dataframe = pd.read_hdf(file_path, 'data')
print(dataframe)

          date        0S        1M        2M        3M        6M        9M  \
0     20050104  0.027393  0.027712  0.028029  0.028344  0.029278  0.030195   
1     20050105  0.025691  0.025962  0.026232  0.026501  0.027300  0.028089   
2     20050106  0.024908  0.025210  0.025510  0.025809  0.026699  0.027577   
3     20050107  0.023603  0.023918  0.024231  0.024542  0.025469  0.026384   
4     20050110  0.024315  0.024628  0.024939  0.025249  0.026170  0.027079   
...        ...       ...       ...       ...       ...       ...       ...   
5080  20250425  0.013377  0.014312  0.014416  0.014468  0.014793  0.014500   
5081  20250427  0.013571  0.014112  0.014566  0.014618  0.014793  0.014500   
5082  20250428  0.013621  0.014112  0.014500  0.014508  0.014697  0.014600   
5083  20250429  0.013624  0.014512  0.014523  0.014850  0.014784  0.014550   
5084  20250430  0.014121  0.014483  0.014494  0.014659  0.014691  0.014500   

            1Y        2Y        3Y  ...        6Y        7Y    

c:\Users\bllos\miniconda3\envs\rqalpha\Lib\site-packages\tables\path.py:146: NaturalNameWarning: object name is not a valid Python identifier: '0S'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
c:\Users\bllos\miniconda3\envs\rqalpha\Lib\site-packages\tables\path.py:146: NaturalNameWarning: object name is not a valid Python identifier: '1M'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
c:\Users\bllos\miniconda3\envs\rqalpha\Lib\site-packages\tables\path.py:146: NaturalNameWarning: object name is not a valid Python identifier: '2M'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will stil

`rqalpha download-bundle` 源码分析
rqalpha.cmds.bundle.download_bundle

代理设置环境变量`RQALPHA_PROXY`


在文件的具体下载逻辑中还支持断点续传，重试次数默认3次
``` python
def download(out, total_length, url):
    retry_interval = 3
    retry_times = 5
    proxy_uri = os.environ.get('RQALPHA_PROXY')

    # 通过click.progressbar进行进度条展示
    # 进度条总长度为`total_length`,即文件大小，同时通过下面的短线续传功能，保障多次重试也能正确展示进度
    with click.progressbar(length=total_length, label=_(u"downloading ...")) as bar:
        for i in range(retry_times):
            try:
                # 通过headers推送bytes=[position]-来告诉服务器从第position的字节位置开始传输，而不是整个文件进行传递
                headers = {'Range': "bytes={}-".format(bar.pos)}
                r = requests.get(url, headers=headers, stream=True, timeout=10, proxies={'http': proxy_uri,
                                                                                         'https': proxy_uri})
                for data in r.iter_content(chunk_size=8192):
                    bar.update(len(data))
                    out.write(data)

                if total_length == bar.pos:
                    return True  # Download complete . exit
            except requests.exceptions.RequestException:
                if i < retry_times - 1:
                    six.print_(_("\nDownload failed, retry in {} seconds.".format(retry_interval)))
                    time.sleep(retry_interval)
                else:
                    raise
```

In [2]:
from rqalpha.cmds.entry import cli
from click.testing import CliRunner

runner = CliRunner()

result = runner.invoke(cli, ['download-bundle', '-d', 'd:/temp/bundle4'])

print(result)

<Result okay>
